In [ ]:
import os
env = {
    'AWS_REGION': 'us-standard',
    'S3_ENDPOINT': 's3.eu-gb.cloud-object-storage.appdomain.cloud',
    'AWS_ACCESS_KEY_ID': 'dcd241d33c7142d6b8a80251e05375fc',
    'AWS_SECRET_ACCESS_KEY': 'd30074e169ee8aa885dba41819c247118e163878f47abc32',
    'S3_USE_HTTPS': '1',
    'S3_VERIFY_SSL': '0',
}
os.environ.update(env)

In [ ]:
import sys
sys.path.append('./gpu-src/')
sys.path.append('/home/lth/lth/gpu-src/')

In [ ]:
import official.resnet.cifar10_main
import official.vgg.cifar10_main
import tensorflow as tf

In [ ]:
# mdir = 's3://REDACTED-data-eu-gb/execution_data/vgg_16_nofc/prune_global_98.2/v4/oneshot_finetune/finetune_0/trial_2/iter_1'
mdir = 'gs://REDACTED/execution_data/resnet20/global_magnitude_20/v2/lottery/prune_4701/trial_1/iter_7'
# mdir = 's3://REDACTED-data/execution_data/vgg_16/prune_global_20/v1/lottery/prune_4701/trial_1/iter_1/'

In [ ]:

clf = tf.estimator.Estimator(
      model_fn=official.resnet.cifar10_main.cifar10_model_fn, 
#       model_fn=official.vgg.cifar10_main.cifar10_model_fn,
        model_dir=mdir, 
#         config=run_config,
#       warm_start_from=warm_start_settings, 
    params={
          'resnet_size': '20',
          'resnet_version': 1,
#           'vgg_size': '16',
          'data_format': 'channels_last',
          'batch_size': 1,
          'loss_scale': 1,
          'dtype': tf.float32,
          'fine_tune': False
      })


In [ ]:
clf.get_variable_names()

In [ ]:
HEIGHT = 32
WIDTH = 32
NUM_CHANNELS = 3
_DEFAULT_IMAGE_BYTES = HEIGHT * WIDTH * NUM_CHANNELS
# The record is the image plus a one-byte label
_RECORD_BYTES = _DEFAULT_IMAGE_BYTES + 1
NUM_CLASSES = 10
_NUM_DATA_FILES = 5


filenames = official.vgg.cifar10_main.get_filenames(True, 's3://REDACTED-data-eu-gb/cifar10')
dataset = tf.data.FixedLengthRecordDataset(filenames, _RECORD_BYTES)

In [ ]:

s = tf.Session()

In [ ]:
z = s.run(dataset.make_one_shot_iterator().get_next())

In [ ]:
a, b = official.vgg.cifar10_main.parse_record(z, True, tf.float32)

In [ ]:
s.run(b)

In [ ]:
d = {x: clf.get_variable_value(x) for x in clf.get_variable_names() if x.endswith('_w')}

In [ ]:
import numpy as np
all_vars = sum(np.prod(x.shape) for (name, x) in d.items())
conv_vars = sum(np.prod(x.shape) for (name, x) in d.items() if 'conv' in name)
fc_vars = sum(np.prod(x.shape) for (name, x) in d.items() if 'fc' in name)
proj_vars = sum(np.prod(x.shape) for (name, x) in d.items() if 'proj' in name)
print(conv_vars/all_vars)

In [ ]:
conv_vars

In [ ]:
np.rollaxis(d['resnet_model/conv_8_m'][..., 0], 2)

In [ ]:
def prune_global_x(names, weights, masks, x, allowed_pruners=['conv']):
    legal_idxs = {i for (i, n) in enumerate(names) if any(pruner in n for pruner in allowed_pruners)}

    legal_names = [n for (i, n) in enumerate(names) if i in legal_idxs]
    legal_weights = [w for (i, w) in enumerate(weights) if i in legal_idxs]
    legal_masks = [m for (i, m) in enumerate(masks) if i in legal_idxs]

    ravel_weights = np.abs(np.concatenate(list(map(np.ravel, legal_weights))))
    ravel_masks = np.concatenate(list(map(np.ravel, legal_masks)))

    threshold = np.percentile(ravel_weights[ravel_masks > 0.5], x)

    for (name, weight, mask) in zip(legal_names, legal_weights, legal_masks):
        old_frac = mask.sum() / np.prod(mask.shape)
        mask[np.abs(weight) < threshold] = 0
        print('{}: {}->{}'.format(
            name,
            old_frac,
            mask.sum() / np.prod(mask.shape),
        ))
    return masks


In [ ]:
kvs = sorted(d.items())

In [ ]:
prune_global_x(sorted({k[:-2] for (k, v) in kvs}), [v for (k, v) in kvs if k.endswith('_w')], [v for (k, v) in kvs if k.endswith('_m')],
               20, ['conv', 'fc'])

In [ ]:
fc

In [ ]:
strs = [
   '''
conv_1: 1.0->0.9560185185185185
conv_2: 1.0->0.8754340277777778
conv_3: 1.0->0.8888888888888888
conv_4: 1.0->0.8802083333333334
conv_5: 1.0->0.8784722222222222
conv_6: 1.0->0.8845486111111112
conv_7: 1.0->0.8532986111111112
conv_8: 1.0->0.8763020833333334
conv_9: 1.0->0.8685980902777778
conv_10: 1.0->0.8491753472222222
conv_11: 1.0->0.8310546875
conv_12: 1.0->0.8505859375
conv_13: 1.0->0.8157552083333334
conv_14: 1.0->0.8477647569444444
conv_15: 1.0->0.833251953125
conv_16: 1.0->0.8277180989583334
conv_17: 1.0->0.7918023003472222
conv_18: 1.0->0.7904188368055556
conv_19: 1.0->0.6271430121527778
''',
'''
conv_1: 1.0->0.9675925925925926
conv_2: 1.0->0.8732638888888888
conv_3: 1.0->0.8793402777777778
conv_4: 1.0->0.8723958333333334
conv_5: 1.0->0.8645833333333334
conv_6: 1.0->0.8780381944444444
conv_7: 1.0->0.85546875
conv_8: 1.0->0.8914930555555556
conv_9: 1.0->0.8684895833333334
conv_10: 1.0->0.8450520833333334
conv_11: 1.0->0.8286675347222222
conv_12: 1.0->0.8421223958333334
conv_13: 1.0->0.8072916666666666
conv_14: 1.0->0.8462456597222222
conv_15: 1.0->0.832763671875
conv_16: 1.0->0.8232150607638888
conv_17: 1.0->0.7918565538194444
conv_18: 1.0->0.7989095052083334
conv_19: 1.0->0.6322699652777778
''',
'''
conv_1: 1.0->0.9675925925925926
conv_2: 1.0->0.8671875
conv_3: 1.0->0.8689236111111112
conv_4: 1.0->0.8797743055555556
conv_5: 1.0->0.8732638888888888
conv_6: 1.0->0.8962673611111112
conv_7: 1.0->0.8706597222222222
conv_8: 1.0->0.8799913194444444
conv_9: 1.0->0.869140625
conv_10: 1.0->0.8470052083333334
conv_11: 1.0->0.8212890625
conv_12: 1.0->0.8468967013888888
conv_13: 1.0->0.8113064236111112
conv_14: 1.0->0.8517252604166666
conv_15: 1.0->0.8357747395833334
conv_16: 1.0->0.8248969184027778
conv_17: 1.0->0.7913953993055556
conv_18: 1.0->0.7920464409722222
conv_19: 1.0->0.6297200520833334
''',
'''
conv_1: 1.0->0.9513888888888888
conv_2: 1.0->0.8797743055555556
conv_3: 1.0->0.8684895833333334
conv_4: 1.0->0.8771701388888888
conv_5: 1.0->0.8936631944444444
conv_6: 1.0->0.8888888888888888
conv_7: 1.0->0.8424479166666666
conv_8: 1.0->0.8851996527777778
conv_9: 1.0->0.8604600694444444
conv_10: 1.0->0.83984375
conv_11: 1.0->0.8240017361111112
conv_12: 1.0->0.8465711805555556
conv_13: 1.0->0.8008897569444444
conv_14: 1.0->0.8428276909722222
conv_15: 1.0->0.8348795572916666
conv_16: 1.0->0.8253038194444444
conv_17: 1.0->0.7944878472222222
conv_18: 1.0->0.7981499565972222
conv_19: 1.0->0.6321885850694444
''',
'''
conv_1: 1.0->0.9328703703703703
conv_2: 1.0->0.8741319444444444
conv_3: 1.0->0.8858506944444444
conv_4: 1.0->0.8802083333333334
conv_5: 1.0->0.8736979166666666
conv_6: 1.0->0.8732638888888888
conv_7: 1.0->0.8723958333333334
conv_8: 1.0->0.8747829861111112
conv_9: 1.0->0.8617621527777778
conv_10: 1.0->0.8416883680555556
conv_11: 1.0->0.8231336805555556
conv_12: 1.0->0.8458116319444444
conv_13: 1.0->0.8037109375
conv_14: 1.0->0.84619140625
conv_15: 1.0->0.8336317274305556
conv_16: 1.0->0.8177354600694444
conv_17: 1.0->0.7928602430555556
conv_18: 1.0->0.8059353298611112
conv_19: 1.0->0.632568359375
''',
'''
conv_1: 1.0->0.9444444444444444
conv_2: 1.0->0.8563368055555556
conv_3: 1.0->0.8823784722222222
conv_4: 1.0->0.8763020833333334
conv_5: 1.0->0.8676215277777778
conv_6: 1.0->0.8702256944444444
conv_7: 1.0->0.859375
conv_8: 1.0->0.8823784722222222
conv_9: 1.0->0.8674045138888888
conv_10: 1.0->0.8434244791666666
conv_11: 1.0->0.830078125
conv_12: 1.0->0.8371310763888888
conv_13: 1.0->0.8107638888888888
conv_14: 1.0->0.8466254340277778
conv_15: 1.0->0.83544921875
conv_16: 1.0->0.8262261284722222
conv_17: 1.0->0.7934027777777778
conv_18: 1.0->0.7945963541666666
conv_19: 1.0->0.6317003038194444
''',
'''
conv_1: 1.0->0.9513888888888888
conv_2: 1.0->0.8589409722222222
conv_3: 1.0->0.8767361111111112
conv_4: 1.0->0.8810763888888888
conv_5: 1.0->0.86328125
conv_6: 1.0->0.8706597222222222
conv_7: 1.0->0.828125
conv_8: 1.0->0.8780381944444444
conv_9: 1.0->0.8614366319444444
conv_10: 1.0->0.8436414930555556
conv_11: 1.0->0.8279079861111112
conv_12: 1.0->0.8366970486111112
conv_13: 1.0->0.8069661458333334
conv_14: 1.0->0.8486870659722222
conv_15: 1.0->0.8377278645833334
conv_16: 1.0->0.8269314236111112
conv_17: 1.0->0.7932671440972222
conv_18: 1.0->0.7948404947916666
conv_19: 1.0->0.6328667534722222
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.875
conv_3: 1.0->0.8615451388888888
conv_4: 1.0->0.8702256944444444
conv_5: 1.0->0.8637152777777778
conv_6: 1.0->0.8836805555555556
conv_7: 1.0->0.8693576388888888
conv_8: 1.0->0.875
conv_9: 1.0->0.8639322916666666
conv_10: 1.0->0.8391927083333334
conv_11: 1.0->0.8238932291666666
conv_12: 1.0->0.8410373263888888
conv_13: 1.0->0.7954644097222222
conv_14: 1.0->0.8434787326388888
conv_15: 1.0->0.8310818142361112
conv_16: 1.0->0.8225911458333334
conv_17: 1.0->0.7940266927083334
conv_18: 1.0->0.8010525173611112
conv_19: 1.0->0.6411675347222222
''',
'''
conv_1: 1.0->0.9629629629629629
conv_2: 1.0->0.8736979166666666
conv_3: 1.0->0.8962673611111112
conv_4: 1.0->0.8719618055555556
conv_5: 1.0->0.8728298611111112
conv_6: 1.0->0.8980034722222222
conv_7: 1.0->0.87109375
conv_8: 1.0->0.8917100694444444
conv_9: 1.0->0.8602430555555556
conv_10: 1.0->0.8395182291666666
conv_11: 1.0->0.8171657986111112
conv_12: 1.0->0.8454861111111112
conv_13: 1.0->0.8202039930555556
conv_14: 1.0->0.8541666666666666
conv_15: 1.0->0.8357747395833334
conv_16: 1.0->0.8256293402777778
conv_17: 1.0->0.7913140190972222
conv_18: 1.0->0.7913411458333334
conv_19: 1.0->0.6284993489583334
''',
'''
conv_1: 1.0->0.9398148148148148
conv_2: 1.0->0.8697916666666666
conv_3: 1.0->0.8797743055555556
conv_4: 1.0->0.8480902777777778
conv_5: 1.0->0.8628472222222222
conv_6: 1.0->0.8580729166666666
conv_7: 1.0->0.8181423611111112
conv_8: 1.0->0.8648003472222222
conv_9: 1.0->0.8572048611111112
conv_10: 1.0->0.8447265625
conv_11: 1.0->0.8172743055555556
conv_12: 1.0->0.8402777777777778
conv_13: 1.0->0.8118489583333334
conv_14: 1.0->0.8501519097222222
conv_15: 1.0->0.8325737847222222
conv_16: 1.0->0.8245985243055556
conv_17: 1.0->0.7963595920138888
conv_18: 1.0->0.7994520399305556
conv_19: 1.0->0.6382378472222222
''',
'''
conv_1: 1.0->0.9513888888888888
conv_2: 1.0->0.8637152777777778
conv_3: 1.0->0.8758680555555556
conv_4: 1.0->0.8515625
conv_5: 1.0->0.8706597222222222
conv_6: 1.0->0.8802083333333334
conv_7: 1.0->0.8667534722222222
conv_8: 1.0->0.8836805555555556
conv_9: 1.0->0.8644748263888888
conv_10: 1.0->0.8382161458333334
conv_11: 1.0->0.8255208333333334
conv_12: 1.0->0.8314887152777778
conv_13: 1.0->0.8006727430555556
conv_14: 1.0->0.84619140625
conv_15: 1.0->0.8362901475694444
conv_16: 1.0->0.8300238715277778
conv_17: 1.0->0.7957356770833334
conv_18: 1.0->0.7964952256944444
conv_19: 1.0->0.629638671875
''',
'''
conv_1: 1.0->0.9699074074074074
conv_2: 1.0->0.8667534722222222
conv_3: 1.0->0.875
conv_4: 1.0->0.8745659722222222
conv_5: 1.0->0.8715277777777778
conv_6: 1.0->0.8923611111111112
conv_7: 1.0->0.8693576388888888
conv_8: 1.0->0.875
conv_9: 1.0->0.8639322916666666
conv_10: 1.0->0.8462456597222222
conv_11: 1.0->0.8262803819444444
conv_12: 1.0->0.8493923611111112
conv_13: 1.0->0.8125
conv_14: 1.0->0.8483615451388888
conv_15: 1.0->0.8354220920138888
conv_16: 1.0->0.8265787760416666
conv_17: 1.0->0.7907172309027778
conv_18: 1.0->0.7960883246527778
conv_19: 1.0->0.6282009548611112
''',
'''
conv_1: 1.0->0.9629629629629629
conv_2: 1.0->0.8828125
conv_3: 1.0->0.8914930555555556
conv_4: 1.0->0.8624131944444444
conv_5: 1.0->0.8689236111111112
conv_6: 1.0->0.8854166666666666
conv_7: 1.0->0.8641493055555556
conv_8: 1.0->0.8723958333333334
conv_9: 1.0->0.8656684027777778
conv_10: 1.0->0.8462456597222222
conv_11: 1.0->0.8280164930555556
conv_12: 1.0->0.8448350694444444
conv_13: 1.0->0.8095703125
conv_14: 1.0->0.8498263888888888
conv_15: 1.0->0.8378634982638888
conv_16: 1.0->0.833740234375
conv_17: 1.0->0.7836100260416666
conv_18: 1.0->0.7828776041666666
conv_19: 1.0->0.6383463541666666
''',
'''
conv_1: 1.0->0.9560185185185185
conv_2: 1.0->0.8854166666666666
conv_3: 1.0->0.8658854166666666
conv_4: 1.0->0.8667534722222222
conv_5: 1.0->0.8715277777777778
conv_6: 1.0->0.8611111111111112
conv_7: 1.0->0.8511284722222222
conv_8: 1.0->0.8797743055555556
conv_9: 1.0->0.8606770833333334
conv_10: 1.0->0.8485243055555556
conv_11: 1.0->0.8250868055555556
conv_12: 1.0->0.8438585069444444
conv_13: 1.0->0.8100043402777778
conv_14: 1.0->0.8513454861111112
conv_15: 1.0->0.8379448784722222
conv_16: 1.0->0.8232150607638888
conv_17: 1.0->0.7920464409722222
conv_18: 1.0->0.7980414496527778
conv_19: 1.0->0.6290147569444444
''',
'''
conv_1: 1.0->0.9606481481481481
conv_2: 1.0->0.8580729166666666
conv_3: 1.0->0.8611111111111112
conv_4: 1.0->0.8715277777777778
conv_5: 1.0->0.8546006944444444
conv_6: 1.0->0.8815104166666666
conv_7: 1.0->0.84375
conv_8: 1.0->0.8687065972222222
conv_9: 1.0->0.8605685763888888
conv_10: 1.0->0.8388671875
conv_11: 1.0->0.8301866319444444
conv_12: 1.0->0.8440755208333334
conv_13: 1.0->0.8153211805555556
conv_14: 1.0->0.8473307291666666
conv_15: 1.0->0.8377821180555556
conv_16: 1.0->0.8268771701388888
conv_17: 1.0->0.7984212239583334
conv_18: 1.0->0.7949761284722222
conv_19: 1.0->0.6272786458333334
''',
'''
conv_1: 1.0->0.9583333333333334
conv_2: 1.0->0.8841145833333334
conv_3: 1.0->0.8619791666666666
conv_4: 1.0->0.8637152777777778
conv_5: 1.0->0.8641493055555556
conv_6: 1.0->0.8845486111111112
conv_7: 1.0->0.8498263888888888
conv_8: 1.0->0.8687065972222222
conv_9: 1.0->0.8669704861111112
conv_10: 1.0->0.8421223958333334
conv_11: 1.0->0.8233506944444444
conv_12: 1.0->0.8407118055555556
conv_13: 1.0->0.7919921875
conv_14: 1.0->0.8470594618055556
conv_15: 1.0->0.8361002604166666
conv_16: 1.0->0.8254665798611112
conv_17: 1.0->0.7996690538194444
conv_18: 1.0->0.8021375868055556
conv_19: 1.0->0.6251898871527778
''',
'''
conv_1: 1.0->0.9652777777777778
conv_2: 1.0->0.8654513888888888
conv_3: 1.0->0.8680555555555556
conv_4: 1.0->0.8645833333333334
conv_5: 1.0->0.8519965277777778
conv_6: 1.0->0.8815104166666666
conv_7: 1.0->0.8658854166666666
conv_8: 1.0->0.8780381944444444
conv_9: 1.0->0.8617621527777778
conv_10: 1.0->0.8375651041666666
conv_11: 1.0->0.8288845486111112
conv_12: 1.0->0.8372395833333334
conv_13: 1.0->0.8091362847222222
conv_14: 1.0->0.8445095486111112
conv_15: 1.0->0.8337131076388888
conv_16: 1.0->0.819091796875
conv_17: 1.0->0.7914496527777778
conv_18: 1.0->0.8045518663194444
conv_19: 1.0->0.6373697916666666
''',
'''
conv_1: 1.0->0.9884259259259259
conv_2: 1.0->0.86328125
conv_3: 1.0->0.8732638888888888
conv_4: 1.0->0.8806423611111112
conv_5: 1.0->0.8680555555555556
conv_6: 1.0->0.8767361111111112
conv_7: 1.0->0.8428819444444444
conv_8: 1.0->0.8767361111111112
conv_9: 1.0->0.8671875
conv_10: 1.0->0.8427734375
conv_11: 1.0->0.8259548611111112
conv_12: 1.0->0.8434244791666666
conv_13: 1.0->0.8146701388888888
conv_14: 1.0->0.8459743923611112
conv_15: 1.0->0.8326822916666666
conv_16: 1.0->0.8222113715277778
conv_17: 1.0->0.7951388888888888
conv_18: 1.0->0.7996148003472222
conv_19: 1.0->0.6314832899305556
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.8719618055555556
conv_3: 1.0->0.8832465277777778
conv_4: 1.0->0.8797743055555556
conv_5: 1.0->0.8715277777777778
conv_6: 1.0->0.8980034722222222
conv_7: 1.0->0.8741319444444444
conv_8: 1.0->0.8830295138888888
conv_9: 1.0->0.8551432291666666
conv_10: 1.0->0.8397352430555556
conv_11: 1.0->0.8245442708333334
conv_12: 1.0->0.8449435763888888
conv_13: 1.0->0.8175998263888888
conv_14: 1.0->0.8482530381944444
conv_15: 1.0->0.8343370225694444
conv_16: 1.0->0.8267686631944444
conv_17: 1.0->0.7941351996527778
conv_18: 1.0->0.7905544704861112
conv_19: 1.0->0.6325141059027778
''',
'''
conv_1: 1.0->0.9629629629629629
conv_2: 1.0->0.8797743055555556
conv_3: 1.0->0.8828125
conv_4: 1.0->0.8927951388888888
conv_5: 1.0->0.8719618055555556
conv_6: 1.0->0.890625
conv_7: 1.0->0.8589409722222222
conv_8: 1.0->0.8763020833333334
conv_9: 1.0->0.8631727430555556
conv_10: 1.0->0.8415798611111112
conv_11: 1.0->0.8225911458333334
conv_12: 1.0->0.8416883680555556
conv_13: 1.0->0.8024088541666666
conv_14: 1.0->0.8443467881944444
conv_15: 1.0->0.83349609375
conv_16: 1.0->0.8243815104166666
conv_17: 1.0->0.7922092013888888
conv_18: 1.0->0.7974175347222222
conv_19: 1.0->0.6358235677083334
''',
'''
conv_1: 1.0->0.9398148148148148
conv_2: 1.0->0.8511284722222222
conv_3: 1.0->0.8684895833333334
conv_4: 1.0->0.8745659722222222
conv_5: 1.0->0.8580729166666666
conv_6: 1.0->0.8719618055555556
conv_7: 1.0->0.8693576388888888
conv_8: 1.0->0.8760850694444444
conv_9: 1.0->0.8665364583333334
conv_10: 1.0->0.8355034722222222
conv_11: 1.0->0.8233506944444444
conv_12: 1.0->0.8510199652777778
conv_13: 1.0->0.8148871527777778
conv_14: 1.0->0.8505859375
conv_15: 1.0->0.8347710503472222
conv_16: 1.0->0.8226725260416666
conv_17: 1.0->0.7912055121527778
conv_18: 1.0->0.7948947482638888
conv_19: 1.0->0.6368543836805556
''',
'''
conv_1: 1.0->0.9537037037037037
conv_2: 1.0->0.8819444444444444
conv_3: 1.0->0.8728298611111112
conv_4: 1.0->0.8771701388888888
conv_5: 1.0->0.8615451388888888
conv_6: 1.0->0.8732638888888888
conv_7: 1.0->0.8289930555555556
conv_8: 1.0->0.8841145833333334
conv_9: 1.0->0.8695746527777778
conv_10: 1.0->0.8470052083333334
conv_11: 1.0->0.8190104166666666
conv_12: 1.0->0.8468967013888888
conv_13: 1.0->0.8056640625
conv_14: 1.0->0.8442925347222222
conv_15: 1.0->0.8316786024305556
conv_16: 1.0->0.8197428385416666
conv_17: 1.0->0.7944064670138888
conv_18: 1.0->0.8002387152777778
conv_19: 1.0->0.6373155381944444
''',
'''
conv_1: 1.0->0.9583333333333334
conv_2: 1.0->0.8797743055555556
conv_3: 1.0->0.8828125
conv_4: 1.0->0.8693576388888888
conv_5: 1.0->0.8693576388888888
conv_6: 1.0->0.88671875
conv_7: 1.0->0.8663194444444444
conv_8: 1.0->0.87890625
conv_9: 1.0->0.8642578125
conv_10: 1.0->0.8382161458333334
conv_11: 1.0->0.8276909722222222
conv_12: 1.0->0.8503689236111112
conv_13: 1.0->0.8067491319444444
conv_14: 1.0->0.8523763020833334
conv_15: 1.0->0.8359375
conv_16: 1.0->0.8234592013888888
conv_17: 1.0->0.7904730902777778
conv_18: 1.0->0.7934841579861112
conv_19: 1.0->0.6330023871527778
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.8580729166666666
conv_3: 1.0->0.8598090277777778
conv_4: 1.0->0.8619791666666666
conv_5: 1.0->0.8641493055555556
conv_6: 1.0->0.8828125
conv_7: 1.0->0.8641493055555556
conv_8: 1.0->0.8821614583333334
conv_9: 1.0->0.8620876736111112
conv_10: 1.0->0.8364800347222222
conv_11: 1.0->0.8234592013888888
conv_12: 1.0->0.8460286458333334
conv_13: 1.0->0.8131510416666666
conv_14: 1.0->0.8495008680555556
conv_15: 1.0->0.8394368489583334
conv_16: 1.0->0.8243543836805556
conv_17: 1.0->0.7926974826388888
conv_18: 1.0->0.7963053385416666
conv_19: 1.0->0.63037109375
''',
'''
conv_1: 1.0->0.9629629629629629
conv_2: 1.0->0.8715277777777778
conv_3: 1.0->0.8658854166666666
conv_4: 1.0->0.8580729166666666
conv_5: 1.0->0.8532986111111112
conv_6: 1.0->0.8815104166666666
conv_7: 1.0->0.8541666666666666
conv_8: 1.0->0.8810763888888888
conv_9: 1.0->0.8581814236111112
conv_10: 1.0->0.8343098958333334
conv_11: 1.0->0.8199869791666666
conv_12: 1.0->0.8408203125
conv_13: 1.0->0.8104383680555556
conv_14: 1.0->0.8477647569444444
conv_15: 1.0->0.8335232204861112
conv_16: 1.0->0.8209364149305556
conv_17: 1.0->0.7982584635416666
conv_18: 1.0->0.8027886284722222
conv_19: 1.0->0.6327582465277778
''',
'''
conv_1: 1.0->0.9606481481481481
conv_2: 1.0->0.8580729166666666
conv_3: 1.0->0.8684895833333334
conv_4: 1.0->0.8763020833333334
conv_5: 1.0->0.8450520833333334
conv_6: 1.0->0.8771701388888888
conv_7: 1.0->0.8441840277777778
conv_8: 1.0->0.8849826388888888
conv_9: 1.0->0.8643663194444444
conv_10: 1.0->0.8396267361111112
conv_11: 1.0->0.8307291666666666
conv_12: 1.0->0.8561197916666666
conv_13: 1.0->0.8128255208333334
conv_14: 1.0->0.8446723090277778
conv_15: 1.0->0.8324652777777778
conv_16: 1.0->0.8223470052083334
conv_17: 1.0->0.7919379340277778
conv_18: 1.0->0.8003200954861112
conv_19: 1.0->0.6334906684027778
''',
'''
conv_1: 1.0->0.9490740740740741
conv_2: 1.0->0.8719618055555556
conv_3: 1.0->0.8815104166666666
conv_4: 1.0->0.8572048611111112
conv_5: 1.0->0.8645833333333334
conv_6: 1.0->0.8741319444444444
conv_7: 1.0->0.8567708333333334
conv_8: 1.0->0.8743489583333334
conv_9: 1.0->0.8624131944444444
conv_10: 1.0->0.8435329861111112
conv_11: 1.0->0.8307291666666666
conv_12: 1.0->0.83984375
conv_13: 1.0->0.8024088541666666
conv_14: 1.0->0.8436957465277778
conv_15: 1.0->0.8396538628472222
conv_16: 1.0->0.8196343315972222
conv_17: 1.0->0.7936197916666666
conv_18: 1.0->0.7998046875
conv_19: 1.0->0.6337890625
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.8645833333333334
conv_3: 1.0->0.8680555555555556
conv_4: 1.0->0.8732638888888888
conv_5: 1.0->0.8697916666666666
conv_6: 1.0->0.8693576388888888
conv_7: 1.0->0.8072916666666666
conv_8: 1.0->0.8815104166666666
conv_9: 1.0->0.8628472222222222
conv_10: 1.0->0.83984375
conv_11: 1.0->0.8260633680555556
conv_12: 1.0->0.8506944444444444
conv_13: 1.0->0.8089192708333334
conv_14: 1.0->0.8436957465277778
conv_15: 1.0->0.8315700954861112
conv_16: 1.0->0.8187391493055556
conv_17: 1.0->0.795654296875
conv_18: 1.0->0.8050944010416666
conv_19: 1.0->0.63623046875
''',
'''
conv_1: 1.0->0.9606481481481481
conv_2: 1.0->0.8784722222222222
conv_3: 1.0->0.8650173611111112
conv_4: 1.0->0.875
conv_5: 1.0->0.86328125
conv_6: 1.0->0.8888888888888888
conv_7: 1.0->0.8532986111111112
conv_8: 1.0->0.8843315972222222
conv_9: 1.0->0.8619791666666666
conv_10: 1.0->0.8377821180555556
conv_11: 1.0->0.8182508680555556
conv_12: 1.0->0.8384331597222222
conv_13: 1.0->0.8051215277777778
conv_14: 1.0->0.8484157986111112
conv_15: 1.0->0.8338216145833334
conv_16: 1.0->0.82373046875
conv_17: 1.0->0.7964138454861112
conv_18: 1.0->0.80078125
conv_19: 1.0->0.63134765625
''',
'''
conv_1: 1.0->0.9537037037037037
conv_2: 1.0->0.8650173611111112
conv_3: 1.0->0.8719618055555556
conv_4: 1.0->0.8615451388888888
conv_5: 1.0->0.8658854166666666
conv_6: 1.0->0.8880208333333334
conv_7: 1.0->0.8845486111111112
conv_8: 1.0->0.8797743055555556
conv_9: 1.0->0.8620876736111112
conv_10: 1.0->0.8408203125
conv_11: 1.0->0.8243272569444444
conv_12: 1.0->0.8458116319444444
conv_13: 1.0->0.8041449652777778
conv_14: 1.0->0.8487413194444444
conv_15: 1.0->0.83056640625
conv_16: 1.0->0.8234592013888888
conv_17: 1.0->0.79248046875
conv_18: 1.0->0.80029296875
conv_19: 1.0->0.6352810329861112
''',
'''
conv_1: 1.0->0.9328703703703703
conv_2: 1.0->0.8606770833333334
conv_3: 1.0->0.8758680555555556
conv_4: 1.0->0.8732638888888888
conv_5: 1.0->0.8667534722222222
conv_6: 1.0->0.8767361111111112
conv_7: 1.0->0.8637152777777778
conv_8: 1.0->0.8717447916666666
conv_9: 1.0->0.8563368055555556
conv_10: 1.0->0.8402777777777778
conv_11: 1.0->0.8182508680555556
conv_12: 1.0->0.8409288194444444
conv_13: 1.0->0.8090277777777778
conv_14: 1.0->0.8426649305555556
conv_15: 1.0->0.8334147135416666
conv_16: 1.0->0.8252224392361112
conv_17: 1.0->0.7954644097222222
conv_18: 1.0->0.7993435329861112
conv_19: 1.0->0.63671875
''',
'''
conv_1: 1.0->0.9490740740740741
conv_2: 1.0->0.8559027777777778
conv_3: 1.0->0.8736979166666666
conv_4: 1.0->0.8771701388888888
conv_5: 1.0->0.8611111111111112
conv_6: 1.0->0.8606770833333334
conv_7: 1.0->0.8493923611111112
conv_8: 1.0->0.8667534722222222
conv_9: 1.0->0.8592664930555556
conv_10: 1.0->0.8436414930555556
conv_11: 1.0->0.8312717013888888
conv_12: 1.0->0.8505859375
conv_13: 1.0->0.8132595486111112
conv_14: 1.0->0.8477647569444444
conv_15: 1.0->0.8347981770833334
conv_16: 1.0->0.8283420138888888
conv_17: 1.0->0.7932671440972222
conv_18: 1.0->0.795654296875
conv_19: 1.0->0.6311577690972222
''',
'''
conv_1: 1.0->0.9560185185185185
conv_2: 1.0->0.8723958333333334
conv_3: 1.0->0.8758680555555556
conv_4: 1.0->0.8624131944444444
conv_5: 1.0->0.8506944444444444
conv_6: 1.0->0.88671875
conv_7: 1.0->0.8667534722222222
conv_8: 1.0->0.8834635416666666
conv_9: 1.0->0.8643663194444444
conv_10: 1.0->0.8420138888888888
conv_11: 1.0->0.8247612847222222
conv_12: 1.0->0.8442925347222222
conv_13: 1.0->0.8095703125
conv_14: 1.0->0.8465711805555556
conv_15: 1.0->0.8333062065972222
conv_16: 1.0->0.8251953125
conv_17: 1.0->0.7973090277777778
conv_18: 1.0->0.7948404947916666
conv_19: 1.0->0.631103515625
''',
'''
conv_1: 1.0->0.9467592592592593
conv_2: 1.0->0.8663194444444444
conv_3: 1.0->0.8745659722222222
conv_4: 1.0->0.87109375
conv_5: 1.0->0.86328125
conv_6: 1.0->0.8914930555555556
conv_7: 1.0->0.8684895833333334
conv_8: 1.0->0.8797743055555556
conv_9: 1.0->0.8665364583333334
conv_10: 1.0->0.8466796875
conv_11: 1.0->0.8264973958333334
conv_12: 1.0->0.8579644097222222
conv_13: 1.0->0.8153211805555556
conv_14: 1.0->0.8541124131944444
conv_15: 1.0->0.8388943142361112
conv_16: 1.0->0.8298611111111112
conv_17: 1.0->0.792724609375
conv_18: 1.0->0.78515625
conv_19: 1.0->0.6242947048611112
''',
'''
conv_1: 1.0->0.9444444444444444
conv_2: 1.0->0.8723958333333334
conv_3: 1.0->0.8815104166666666
conv_4: 1.0->0.8819444444444444
conv_5: 1.0->0.8836805555555556
conv_6: 1.0->0.8862847222222222
conv_7: 1.0->0.8624131944444444
conv_8: 1.0->0.8780381944444444
conv_9: 1.0->0.8739149305555556
conv_10: 1.0->0.8361545138888888
conv_11: 1.0->0.8285590277777778
conv_12: 1.0->0.8460286458333334
conv_13: 1.0->0.8169487847222222
conv_14: 1.0->0.8511284722222222
conv_15: 1.0->0.8358289930555556
conv_16: 1.0->0.8203125
conv_17: 1.0->0.7903645833333334
conv_18: 1.0->0.7967122395833334
conv_19: 1.0->0.6292588975694444
''',
'''
conv_1: 1.0->0.9490740740740741
conv_2: 1.0->0.8702256944444444
conv_3: 1.0->0.8689236111111112
conv_4: 1.0->0.8667534722222222
conv_5: 1.0->0.8641493055555556
conv_6: 1.0->0.8689236111111112
conv_7: 1.0->0.8541666666666666
conv_8: 1.0->0.8778211805555556
conv_9: 1.0->0.8640407986111112
conv_10: 1.0->0.8446180555555556
conv_11: 1.0->0.8236762152777778
conv_12: 1.0->0.8514539930555556
conv_13: 1.0->0.8098958333333334
conv_14: 1.0->0.8452690972222222
conv_15: 1.0->0.8369683159722222
conv_16: 1.0->0.8281521267361112
conv_17: 1.0->0.793212890625
conv_18: 1.0->0.796142578125
conv_19: 1.0->0.6277940538194444
''',
'''
conv_1: 1.0->0.9398148148148148
conv_2: 1.0->0.8797743055555556
conv_3: 1.0->0.8667534722222222
conv_4: 1.0->0.8697916666666666
conv_5: 1.0->0.8559027777777778
conv_6: 1.0->0.8927951388888888
conv_7: 1.0->0.8550347222222222
conv_8: 1.0->0.8802083333333334
conv_9: 1.0->0.8634982638888888
conv_10: 1.0->0.8386501736111112
conv_11: 1.0->0.8190104166666666
conv_12: 1.0->0.8309461805555556
conv_13: 1.0->0.8002387152777778
conv_14: 1.0->0.8487413194444444
conv_15: 1.0->0.8355577256944444
conv_16: 1.0->0.8216688368055556
conv_17: 1.0->0.7964138454861112
conv_18: 1.0->0.7991265190972222
conv_19: 1.0->0.6360677083333334
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.8502604166666666
conv_3: 1.0->0.87109375
conv_4: 1.0->0.8619791666666666
conv_5: 1.0->0.8719618055555556
conv_6: 1.0->0.8962673611111112
conv_7: 1.0->0.8663194444444444
conv_8: 1.0->0.8862847222222222
conv_9: 1.0->0.8667534722222222
conv_10: 1.0->0.8473307291666666
conv_11: 1.0->0.8272569444444444
conv_12: 1.0->0.8465711805555556
conv_13: 1.0->0.8147786458333334
conv_14: 1.0->0.8493381076388888
conv_15: 1.0->0.8402777777777778
conv_16: 1.0->0.827392578125
conv_17: 1.0->0.7884114583333334
conv_18: 1.0->0.7915852864583334
conv_19: 1.0->0.6284450954861112
''',
'''
conv_1: 1.0->0.9675925925925926
conv_2: 1.0->0.8637152777777778
conv_3: 1.0->0.8784722222222222
conv_4: 1.0->0.8576388888888888
conv_5: 1.0->0.8736979166666666
conv_6: 1.0->0.8780381944444444
conv_7: 1.0->0.8580729166666666
conv_8: 1.0->0.8760850694444444
conv_9: 1.0->0.8637152777777778
conv_10: 1.0->0.8422309027777778
conv_11: 1.0->0.8191189236111112
conv_12: 1.0->0.8484157986111112
conv_13: 1.0->0.8154296875
conv_14: 1.0->0.8482530381944444
conv_15: 1.0->0.8371039496527778
conv_16: 1.0->0.8229709201388888
conv_17: 1.0->0.7922634548611112
conv_18: 1.0->0.7998318142361112
conv_19: 1.0->0.6294487847222222
''',
'''
conv_1: 1.0->0.9351851851851852
conv_2: 1.0->0.8810763888888888
conv_3: 1.0->0.8828125
conv_4: 1.0->0.8719618055555556
conv_5: 1.0->0.8619791666666666
conv_6: 1.0->0.8871527777777778
conv_7: 1.0->0.8624131944444444
conv_8: 1.0->0.8819444444444444
conv_9: 1.0->0.8646918402777778
conv_10: 1.0->0.8466796875
conv_11: 1.0->0.8192274305555556
conv_12: 1.0->0.8240017361111112
conv_13: 1.0->0.7861328125
conv_14: 1.0->0.8446180555555556
conv_15: 1.0->0.837646484375
conv_16: 1.0->0.8167588975694444
conv_17: 1.0->0.7916124131944444
conv_18: 1.0->0.802978515625
conv_19: 1.0->0.6435275607638888
''',
'''
conv_1: 1.0->0.9421296296296297
conv_2: 1.0->0.8728298611111112
conv_3: 1.0->0.8810763888888888
conv_4: 1.0->0.8958333333333334
conv_5: 1.0->0.8676215277777778
conv_6: 1.0->0.8858506944444444
conv_7: 1.0->0.8628472222222222
conv_8: 1.0->0.8843315972222222
conv_9: 1.0->0.8580729166666666
conv_10: 1.0->0.8287760416666666
conv_11: 1.0->0.8113064236111112
conv_12: 1.0->0.8512369791666666
conv_13: 1.0->0.8093532986111112
conv_14: 1.0->0.8464626736111112
conv_15: 1.0->0.835693359375
conv_16: 1.0->0.828369140625
conv_17: 1.0->0.7928602430555556
conv_18: 1.0->0.7964409722222222
conv_19: 1.0->0.6318630642361112
''',
'''
conv_1: 1.0->0.9675925925925926
conv_2: 1.0->0.8598090277777778
conv_3: 1.0->0.8732638888888888
conv_4: 1.0->0.8650173611111112
conv_5: 1.0->0.8719618055555556
conv_6: 1.0->0.8684895833333334
conv_7: 1.0->0.8706597222222222
conv_8: 1.0->0.8745659722222222
conv_9: 1.0->0.8607855902777778
conv_10: 1.0->0.8347439236111112
conv_11: 1.0->0.8182508680555556
conv_12: 1.0->0.8351779513888888
conv_13: 1.0->0.7994791666666666
conv_14: 1.0->0.8484157986111112
conv_15: 1.0->0.8302137586805556
conv_16: 1.0->0.8187120225694444
conv_17: 1.0->0.7974989149305556
conv_18: 1.0->0.8035481770833334
conv_19: 1.0->0.6417100694444444
''',
'''
conv_1: 1.0->0.9490740740740741
conv_2: 1.0->0.8498263888888888
conv_3: 1.0->0.86328125
conv_4: 1.0->0.8498263888888888
conv_5: 1.0->0.8572048611111112
conv_6: 1.0->0.8611111111111112
conv_7: 1.0->0.8602430555555556
conv_8: 1.0->0.8741319444444444
conv_9: 1.0->0.8693576388888888
conv_10: 1.0->0.8487413194444444
conv_11: 1.0->0.8238932291666666
conv_12: 1.0->0.8472222222222222
conv_13: 1.0->0.8087022569444444
conv_14: 1.0->0.8445638020833334
conv_15: 1.0->0.8362087673611112
conv_16: 1.0->0.8271213107638888
conv_17: 1.0->0.796630859375
conv_18: 1.0->0.7939724392361112
conv_19: 1.0->0.6312662760416666
''',
   
]

In [ ]:
z = [dictit(i) for i in strs]
res = {i: 0 for i in range(1, 20)}
for i in z:
    for j in range(1, 20):
        res[j] += i[j]
        
for j in range(1, 20):
    res[j] /= len(z)

In [ ]:
{k: (1-v)/2 for (k,v) in res.items()}